In [4]:
import pandas as pd
import seaborn as sns
import sqlite3
import os
import sys
import sklearn

In [5]:
db_path = os.path.join(os.getcwd(),'data/game_database.sqlite3')
con = sqlite3.connect(db_path)
print(con)

In [6]:
query = """
SELECT * FROM metadata;
"""
meta_df = pd.read_sql(query, con)

In [7]:
def get_winner(row):
    if row["lras"] == 1:
        return row["player_1_tag"]
    elif row["lras"] == 0:
        return row["player_2_tag"]
    if row["player_1_end_stocks"] == 0:
        return row["player_2_tag"]
    elif (row["player_1_end_stocks"] == row["player_2_end_stocks"]) & (row["player_1_end_dmg"] > row["player_2_end_dmg"]):
        return row["player_2_tag"]
    else:
        return row["player_1_tag"]        
    
meta_df["winner"] = meta_df.apply(get_winner,axis=1)
def winner_char(row):
    if row["lras"] == 1:
        return row["player_1_char"]
    elif row["lras"] == 0:
        return row["player_2_char"]
    if row["player_1_end_stocks"] == 0:
        return row["player_2_char"]
    elif (row["player_1_end_stocks"] == row["player_2_end_stocks"]) & (row["player_1_end_dmg"] > row["player_2_end_dmg"]):
        return row["player_2_char"]
    else:
        return row["player_1_char"]  
meta_df["winner_char"] = meta_df.apply(winner_char,axis=1)

In [8]:
meta_df.head(10)

,date,path,duration_frames,player_1_tag,player_1_char,player_1_end_stocks,player_1_end_dmg,player_2_tag,player_2_char,player_2_end_stocks,player_2_end_dmg,stage,end_method,lras,winner,winner_char
0,2022-01-01 15:12:20+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,10231,TIGE#649,FALCO,1,74.330002,YARI#105,FALCO,0,0.000000,FINAL_DESTINATION,GAME,NaN,TIGE#649,FALCO
1,2022-01-01 15:02:36+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,12711,TIGE#649,FALCO,0,111.620003,YARI#105,FALCO,2,8.190000,POKEMON_STADIUM,GAME,NaN,YARI#105,FALCO
2,2022-01-01 15:06:15+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,11547,TIGE#649,JIGGLYPUFF,0,111.726501,YARI#105,JIGGLYPUFF,1,106.099998,YOSHIS_STORY,GAME,NaN,YARI#105,JIGGLYPUFF
3,2022-01-01 15:09:33+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,124,TIGE#649,JIGGLYPUFF,4,0.000000,YARI#105,JIGGLYPUFF,4,0.000000,DREAM_LAND_N64,NO_CONTEST,0.0,YARI#105,JIGGLYPUFF
4,2022-01-01 15:09:42+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,9006,TIGE#649,FALCO,0,96.089996,YARI#105,FALCO,3,70.120003,BATTLEFIELD,GAME,NaN,YARI#105,FALCO
5,2022-01-01 15:15:18+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,22040,TIGE#649,JIGGLYPUFF,1,88.470001,YARI#105,JIGGLYPUFF,0,154.669983,FOUNTAIN_OF_DREAMS,GAME,NaN,TIGE#649,JIGGLYPUFF
6,2022-01-01 15:21:35+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,16572,TIGE#649,JIGGLYPUFF,1,73.180000,YARI#105,JIGGLYPUFF,0,39.000000,FOUNTAIN_OF_DREAMS,GAME,NaN,TIGE#649,JIGGLYPUFF
7,2022-01-01 15:26:56+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,12877,2#2,MARIO,1,96.160004,YARI#105,MARIO,0,125.360001,YOSHIS_STORY,GAME,NaN,2#2,MARIO
8,2022-01-01 15:30:46+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,9345,YARI#105,FOX,0,119.459999,THRU#668,FOX,3,75.979996,DREAM_LAND_N64,GAME,NaN,THRU#668,FOX
9,2022-01-01 15:33:28+00:00,/media/beastman/36E8AEA6E8AE63B9/Users/newco/D...,10129,YARI#105,JIGGLYPUFF,0,122.569992,THRU#668,JIGGLYPUFF,2,69.839996,FINAL_DESTINATION,GAME,NaN,THRU#668,JIGGLYPUFF


In [9]:
print(len(meta_df[meta_df["winner"] == "YARI#105"]) / len(meta_df))

0.5806451612903226


In [10]:
# SELECT 
# FROM
# WHERE/HAVING
# GROUP BY
# ORDER BY
# find the average frame duration for games where YARI#105 is the winner, grouped by character in descending order
query = """
    WITH test_df AS (
    SELECT *,
        CASE
            WHEN lras = 1
                THEN player_1_char
            WHEN lras = 0
                THEN player_2_char
            WHEN player_1_end_stocks = 0
                THEN player_2_char
            ELSE player_1_char 
            END AS winner_char,
        CASE
            WHEN lras = 1
                THEN player_1_tag
            WHEN lras = 0
                THEN player_2_tag
            WHEN player_1_end_stocks = 0
                THEN player_2_tag
            ELSE player_1_tag
            END AS winner_tag
    FROM metadata
    )
    
    SELECT
        winner_char,
        AVG (duration_frames) AS avg_duration_frames,
        COUNT(duration_frames) AS n_games
    FROM
        test_df
    WHERE 1=1
        AND winner_tag = 'YARI#105'        
    GROUP BY
        winner_char
    ORDER BY
        AVG(duration_frames) desc
        """
pd.read_sql(query,con)

,winner_char,avg_duration_frames,n_games
0,PEACH,12504.000000,1
1,MEWTWO,12020.000000,1
2,CAPTAIN_FALCON,11640.000000,1
3,FOX,8767.428571,7
4,FALCO,8446.500000,4
5,SHEIK,7819.000000,2
6,JIGGLYPUFF,5835.500000,2


In [11]:
con.close()